# 环境准备

In [ ]:
#@title ## Clone 代码，可能会提示重启 Runtime，点击重启即可
!git clone https://github.com/noisyle/Bert-VITS2.git
%cd Bert-VITS2
!git checkout dev
!pip install -r requirements.txt

In [ ]:
#@title ## 下载 BERT 模型
%cd /content/Bert-VITS2
!cd monotonic_align && python setup.py build_ext --inplace
!wget https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin -O bert/chinese-roberta-wwm-ext-large/pytorch_model.bin

# 训练

In [ ]:
#@title ## 数据清洗
#@markdown - 首先将训练用的 wav 格式音频文件放置到 workspace/raw_audio 目录下。可以通过 Google Drive 上传音频文件。
#@markdown - 文件命名规则为 <说话人>_<任意数字>.wav，如 paimeng_1.wav、paimeng_2.wav。
#@markdown - 可以同时放置多个说话人的多个音频文件。

!python scripts/denoise_audio.py
!python scripts/transcribe_audio.py
!python resample.py
!python preprocess_text.py
!python bert_gen.py

In [ ]:
#@title ## 开始训练
!python train_ms.py -m test

# 推理

In [14]:
#@title ## (Optional) 挂载 Google Drive，通过 Google Drive 上传模型权重。
#@markdown ### 需提前将训练好的模型上传到 Google Drive。
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/test.zip -d logs/test/

In [ ]:
#@title ## 启动 WebUI
!python webui.py -c logs/test/config.json -m logs/test/G_26000.pth --share